In [4]:
import nibabel as nib
import numpy as np

sphere = "data/MacaqueYerkes19_v1.2.L.sphere.32k_fs_LR.surf.gii"
sphere = nib.load(sphere)
coordinates, vertices = sphere.agg_data()

x = "data/Human_GC1_to_Monkey.L.32k_fs_LR.shape.gii"
x = nib.load(x).agg_data()
y = "data/Monkey_GC2_nPDF.L.32k_fs_LR.shape.gii"
y = nib.load(y).agg_data()

In [5]:
coordinates

array([[ 85.06509  ,   0.       ,  52.573116 ],
       [  0.       , -52.573116 ,  85.06509  ],
       [-85.06509  ,   0.       ,  52.573116 ],
       ...,
       [ -1.9288353, -48.54366  , -87.405914 ],
       [ -3.834274 , -49.721283 , -86.67809  ],
       [ -1.921215 , -50.295624 , -86.409836 ]], dtype=float32)

In [6]:
np.sqrt((coordinates ** 2).sum(axis=1))[:, np.newaxis]

array([[100.00001],
       [100.00001],
       [100.00001],
       ...,
       [100.00001],
       [100.     ],
       [100.     ]], dtype=float32)

In [7]:
v = coordinates / np.sqrt((coordinates ** 2).sum(axis=1))[:, np.newaxis]

In [8]:
r = np.zeros(v.shape[0])

vals = np.logical_and(x != 0, y != 0)
vals

array([ True,  True,  True, ...,  True,  True,  True])

In [9]:
v

array([[ 0.8506508 ,  0.        ,  0.52573115],
       [ 0.        , -0.52573115,  0.8506508 ],
       [-0.8506508 ,  0.        ,  0.52573115],
       ...,
       [-0.01928835, -0.48543656, -0.8740591 ],
       [-0.03834274, -0.49721283, -0.86678094],
       [-0.01921215, -0.5029562 , -0.86409837]], dtype=float32)

In [5]:
cos_angle = v @ v[0]
cos_angle

array([ 1.0000001 ,  0.44721365, -0.44721365, ..., -0.47592777,
       -0.48831   , -0.47062624], dtype=float32)

In [6]:
angle = np.arccos(np.clip(cos_angle, -1, 1))
angle

array([0.       , 1.1071486, 2.034444 , ..., 2.066815 , 2.0809484,
       2.0607967], dtype=float32)

In [7]:
j = (np.degrees(angle) < 30) & vals

r[j] = 1
# 将数据格式转换为int32
r = r.astype(np.int32)
new_gifti = nib.gifti.GiftiDataArray(r)
new_gifti = nib.gifti.GiftiImage(darrays=[new_gifti])
nib.save(new_gifti, "test.func.gii")

In [17]:
def surflocalcorr(x, y, sph, a=30, method="pearsonr"):
    if isinstance(x, str):
        x = nib.load(x)
    x = x.agg_data()
    if isinstance(y, str):
        y = nib.load(y)
    y = y.agg_data()
    if isinstance(sph, str):
        sph = nib.load(sph)
    coordinates, _ = sph.agg_data()
    v = coordinates / np.sqrt((coordinates ** 2).sum(axis=1))[:, np.newaxis]
    r = np.zeros(v.shape[0])

    # no values
    #vals = np.logical_and(x != 0, y != 0)

    for i in range(v.shape[0]):
        cos_angle = v @ v[i]
        angle = np.arccos(np.clip(cos_angle, -1, 1))
        j = (np.degrees(angle) < a)
        # Ensure enough values for stable correlation
        assert np.sum(j) > 30, f"Not enough values for stable correlation, try increasing the angle {i}"
        if method == "pearsonr":
            r[i] = np.corrcoef(x[j], y[j])[0, 1]
        elif method == "spearmanr":
            pass

    #r[~vals] = 0
    return r

In [22]:

sphere = "data/MacaqueYerkes19_v1.2.L.sphere.32k_fs_LR.surf.gii"
x = "data/Human_GC1_to_Monkey.L.32k_fs_LR.shape.gii"
y = "data/Monkey_GC2_nPDF.L.32k_fs_LR.shape.gii"

corr = surflocalcorr(x, y, nib.load(sphere), a=30, method="pearsonr")

In [20]:
corr

array([        nan,  0.53258119,  0.27790825, ...,  0.02052593,
       -0.01120528,  0.02543244])

In [35]:
import nibabel as nib
import numpy as np
from pathlib import PosixPath

def local_corr(x, y, coor, a, method="spearmanr"):
    """
    Calculate the local correlation between two arrays.

    Parameters:
    - x (ndarray): First input array.
    - y (ndarray): Second input array.
    - coor (ndarray): Array of coordinates.
    - a (float): Angle threshold in degrees.
    - method (str): Correlation method to use. Default is "spearmanr".

    Returns:
    - r (ndarray): Array of local correlation values.
    """
    v = coor / np.sqrt((coor**2).sum(axis=1))[:, np.newaxis]
    r = np.zeros(v.shape[0])
    # no values
    vals = np.logical_and(
        ~np.isnan(x), ~np.isnan(y)
    )

    for i in range(v.shape[0]):
        cos_angle = v @ v[i]
        angle = np.arccos(np.clip(cos_angle, -1, 1))
        j = (np.degrees(angle) < a) & vals
        # Ensure enough values for stable correlation
        assert np.sum(j) > 30, "Not enough data points for correlation calculation"
        if method == "pearsonr":
            r[i] = np.corrcoef(x[j], y[j])[0, 1]

    r[~vals] = np.nan
    return r


def surflocalcorr(x, y, sph, a=30, method="spearmanr", return_gifti=False):
    """
    Calculate the surface local correlation between two input datasets.

    Parameters:
    x (str or nibabel.gifti.GiftiImage): Input dataset x. If a string is provided, it is assumed to be the path to the dataset and will be loaded using nibabel.
    y (str or nibabel.gifti.GiftiImage): Input dataset y. If a string is provided, it is assumed to be the path to the dataset and will be loaded using nibabel.
    sph (str or nibabel.gifti.GiftiImage): Spherical coordinates. If a string is provided, it is assumed to be the path to the dataset and will be loaded using nibabel.
    a (int, optional): Parameter a. Defaults to 30.
    method (str, optional): Correlation method. Can be pearsonr or spearmanr. Defaults to 'spearmanr'.
    return_gifti (bool, optional): Whether to return the result as a nibabel.gifti.GiftiImage object. Defaults to False.

    Returns:
    numpy.ndarray or nibabel.gifti.GiftiImage: Array of local correlation values. If return_gifti is True, the result is returned as a nibabel.gifti.GiftiImage object.

    """
    if isinstance(x, PosixPath):
        x = nib.load(x)
    if isinstance(x, str):
        x = nib.load(x)
    x = x.agg_data()
    if isinstance(y, PosixPath):
        y = nib.load(y)
    if isinstance(y, str):
        y = nib.load(y)
    y = y.agg_data()
    if isinstance(sph, PosixPath):
        sph = nib.load(sph)
    if isinstance(sph, str):
        sph = nib.load(sph)
    coordinates, _ = sph.agg_data()
    assert method in [
        "pearsonr",
        "spearmanr",
    ], "Invalid method. Must be 'pearsonr' or 'spearmanr'."
    corr = local_corr(x, y, coordinates, a, method)
    # 保存到Gifti对象
    if return_gifti:
        corr = corr.astype(np.float32)
        corr_gifti = nib.gifti.GiftiDataArray(corr)
        corr_gifti = nib.gifti.GiftiImage(darrays=[corr_gifti])
        return corr_gifti

    return corr

sphere = "data/MacaqueYerkes19_v1.2.L.sphere.32k_fs_LR.surf.gii"
x = "data/Human_GC1_to_Monkey.L.32k_fs_LR.shape.gii"
y = "data/Monkey_GC2_nPDF.L.32k_fs_LR.shape.gii"
corr = surflocalcorr(x, y, sphere, a=40, method="pearsonr", return_gifti=True)
corr.to_filename("test_2.func.gii")